# 🛡️ Data Leakage Prevention - Complete Implementation Guide

## Overview
This notebook demonstrates comprehensive data leakage prevention for QSAR models.

### Critical Issues Addressed:
1. ✅ **Scaffold-based splitting** (not random)
2. ✅ **Duplicate & near-duplicate removal**
3. ✅ **Feature scaling on train only**
4. ✅ **Proper cross-validation**
5. ✅ **Target leakage prevention**
6. ✅ **Similarity analysis**
7. ✅ **Applicability domain**

---

## Step 1: Import Libraries and Load Data

In [ ]:
# Import data leakage prevention utilities from the framework
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem

# Auto-detect framework path (works when cloned from GitHub)
# This finds the 'src' directory relative to the notebook location
current_dir = os.path.dirname(os.path.abspath('__file__')) if '__file__' in dir() else os.getcwd()
repo_root = os.path.abspath(os.path.join(current_dir, '..'))
framework_path = os.path.join(repo_root, 'src')

# Add framework to path if it exists
if os.path.exists(framework_path) and framework_path not in sys.path:
    sys.path.insert(0, framework_path)
    print(f"✓ Framework loaded from: {framework_path}")
else:
    print(f"⚠ Warning: Framework path not found at {framework_path}")
    print("  Make sure you're running from the notebooks/ folder in the cloned repo")

# Import core utilities for data processing
from utils.qsar_utils_no_leakage import QSARDataProcessor

# Import validation modules
from qsar_validation.splitting_strategies import AdvancedSplitter
from qsar_validation.feature_scaling import FeatureScaler
from qsar_validation.feature_selection import FeatureSelector
from qsar_validation.dataset_quality_analysis import DatasetQualityAnalyzer
from qsar_validation.performance_validation import PerformanceValidator
from qsar_validation.activity_cliffs_detection import ActivityCliffsDetector

# Set random seed for reproducibility
np.random.seed(42)

print("✓ Libraries imported successfully")
print("✅ Framework v4.1.0 - Multi-Library Support")
print("\n" + "="*70)
print("QSAR VALIDATION FRAMEWORK - MODULES LOADED:")
print("  ✓ QSARDataProcessor - Duplicate removal & data processing")
print("  ✓ AdvancedSplitter - Scaffold-based splitting")
print("  ✓ FeatureScaler - Proper feature scaling (fit on train only)")
print("  ✓ FeatureSelector - Feature selection to prevent overfitting")
print("  ✓ DatasetQualityAnalyzer - Dataset representativeness checks")
print("  ✓ PerformanceValidator - Cross-validation & metrics")
print("  ✓ ActivityCliffsDetector - Activity cliff analysis")
print("="*70)

In [ ]:
# Load your dataset
# Replace this path with your actual data path
df = pd.read_excel('/content/drive/MyDrive/DrRoyRationalDesign/Input of triazole and cysteine_datasheet.xlsx')

print(f"Original dataset: {len(df)} rows")
df.head()

## Step 2: Data Cleaning - Prevent Leakage from Duplicates

### ⚠️ Common Mistake:
- Random split with duplicates → same molecule in train AND test

### ✅ Solution:
- Canonicalize SMILES
- Remove exact duplicates BEFORE splitting
- Average replicate measurements

In [ ]:
# Initialize processor
processor = QSARDataProcessor(
    smiles_col='Canonical SMILES',
    target_col='IC50 uM'
)

# Step 1: Canonicalize SMILES
df = processor.canonicalize_smiles(df)

# Step 2: Remove exact duplicates (average replicates)
df = processor.remove_duplicates(df, strategy='average')

# Step 3: Keep only numeric IC50 values
mask = pd.to_numeric(df["IC50 uM"], errors="coerce").notna()
df = df[mask].reset_index(drop=True)

print(f"\n✓ Clean dataset: {len(df)} unique molecules with valid IC50 values")

## Step 3: Scaffold-Based Splitting - Critical for QSAR!

### ⚠️ Common Mistake:
```python
# DON'T DO THIS - Random split causes leakage!
train_test_split(X, y, test_size=0.2, random_state=42)
```

### ✅ Solution: Scaffold-based split
- Use Bemis-Murcko scaffolds
- Entire scaffold in train OR test (never both)
- More realistic performance estimation

In [ ]:
# Initialize scaffold splitter from framework
splitter = AdvancedSplitter()

# Perform scaffold-based split
splits = splitter.scaffold_split(
    df,
    smiles_col='Canonical SMILES',
    target_col='IC50 uM',
    test_size=0.2,
    val_size=0.1
)

train_idx = splits['train_idx']
val_idx = splits['val_idx']
test_idx = splits['test_idx']

print(f"\n✓ Scaffold-based split completed:")
print(f"   Training: {len(train_idx)} molecules")
print(f"   Validation: {len(val_idx)} molecules")
print(f"   Test: {len(test_idx)} molecules")

## Step 4: Remove Near-Duplicates Between Splits

### ✅ Check for molecules with Tanimoto similarity ≥ 0.95

In [ ]:
# Remove near-duplicates (Tanimoto >= 0.95)
train_idx, test_idx = processor.remove_near_duplicates(
    df,
    train_idx,
    test_idx,
    threshold=0.95
)

print(f"\n✓ After near-duplicate removal:")
print(f"   Training: {len(train_idx)} molecules")
print(f"   Test: {len(test_idx)} molecules")

## Step 5: Analyze Train-Test Similarity

### 📊 Important for reviewers:
- Shows applicability domain
- Demonstrates no data leakage
- Realistic performance expectations

In [ ]:
# Analyze similarity between train and test
similarity_stats = processor.analyze_similarity(df, train_idx, test_idx)

# Plot similarity distribution
plot_similarity_distribution(similarity_stats, save_path='similarity_distribution.png')

## Step 6: Create Splits and Check for Leakage

In [ ]:
# Create train/val/test dataframes
train_df = df.iloc[train_idx].reset_index(drop=True)
val_df = df.iloc[val_idx].reset_index(drop=True)
test_df = df.iloc[test_idx].reset_index(drop=True)

# CRITICAL CHECK: Ensure no SMILES overlap
train_smiles = set(train_df['Canonical SMILES'])
val_smiles = set(val_df['Canonical SMILES'])
test_smiles = set(test_df['Canonical SMILES'])

overlap_train_test = train_smiles & test_smiles
overlap_train_val = train_smiles & val_smiles
overlap_val_test = val_smiles & test_smiles

print("\n🔍 Checking for SMILES overlap (should all be 0):")
print(f"   Train-Test overlap: {len(overlap_train_test)} molecules")
print(f"   Train-Val overlap: {len(overlap_train_val)} molecules")
print(f"   Val-Test overlap: {len(overlap_val_test)} molecules")

if overlap_train_test or overlap_train_val or overlap_val_test:
    print("\n⚠️  WARNING: DATA LEAKAGE DETECTED!")
else:
    print("\n✅ No data leakage detected - splits are clean!")

## Step 7: Feature Generation (Example with Circular Fingerprints)

### Important: Generate features AFTER splitting

In [ ]:
def generate_circular_fingerprints(smiles_list, radius=2, n_bits=1024):
    """
    Generate Morgan (circular) fingerprints.
    """
    fps = []
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, n_bits)
            fps.append(list(fp))
        else:
            fps.append([0] * n_bits)
    return np.array(fps)

# Generate fingerprints for each split
print("Generating circular fingerprints...")
X_train = generate_circular_fingerprints(train_df['Canonical SMILES'])
X_val = generate_circular_fingerprints(val_df['Canonical SMILES'])
X_test = generate_circular_fingerprints(test_df['Canonical SMILES'])

# Get targets
y_train = train_df['IC50 uM'].values.astype(float)
y_val = val_df['IC50 uM'].values.astype(float)
y_test = test_df['IC50 uM'].values.astype(float)

print(f"✓ Features generated:")
print(f"   X_train: {X_train.shape}")
print(f"   X_val: {X_val.shape}")
print(f"   X_test: {X_test.shape}")

## Step 8: Target Transformation - BEFORE Scaling

### ⚠️ Common Mistake:
```python
# DON'T normalize target using full dataset statistics!
df['IC50_norm'] = (df['IC50'] - df['IC50'].mean()) / df['IC50'].std()
```

### ✅ Solution:
- Transform target (e.g., pIC50, log) BEFORE splitting
- OR fit scaler on train only, then transform val/test

In [ ]:
# Transform to pIC50 (common in QSAR)
def to_pIC50(ic50_uM):
    """Convert IC50 (μM) to pIC50 = -log10(IC50 in M)"""
    return -np.log10(ic50_uM * 1e-6)

y_train_transformed = to_pIC50(y_train)
y_val_transformed = to_pIC50(y_val)
y_test_transformed = to_pIC50(y_test)

print("✓ Target transformed to pIC50")
print(f"   Train: {y_train_transformed.shape}, range [{y_train_transformed.min():.2f}, {y_train_transformed.max():.2f}]")
print(f"   Val: {y_val_transformed.shape}, range [{y_val_transformed.min():.2f}, {y_val_transformed.max():.2f}]")
print(f"   Test: {y_test_transformed.shape}, range [{y_test_transformed.min():.2f}, {y_test_transformed.max():.2f}]")

## Step 9: Feature Scaling - FIT ON TRAIN ONLY! 🚨

### ⚠️ CRITICAL MISTAKE (causes leakage):
```python
# NEVER DO THIS!
scaler = StandardScaler()
X_all_scaled = scaler.fit_transform(X_all)  # Uses test statistics!
```

### ✅ CORRECT WAY:

In [ ]:
from sklearn.preprocessing import StandardScaler

# FIT scaler on TRAINING data ONLY
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# TRANSFORM (not fit_transform!) validation and test
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("✅ CORRECT: Scaler fitted on training data only")
print("✅ CORRECT: Validation and test transformed using training statistics")
print(f"\n   X_train_scaled: {X_train_scaled.shape}")
print(f"   X_val_scaled: {X_val_scaled.shape}")
print(f"   X_test_scaled: {X_test_scaled.shape}")

## Step 10: Scaffold-Based Cross-Validation

### ⚠️ Common Mistake:
```python
# Random K-Fold - causes leakage!
KFold(n_splits=5, shuffle=True)
```

### ✅ Solution: Scaffold-based K-Fold

In [ ]:
# Create scaffold-based K-fold splits
cv_splits = splitter.scaffold_kfold(train_df, n_splits=5, random_state=42)

print(f"\n✓ Created {len(cv_splits)} scaffold-based CV folds")
print("\nUse these splits for:")
print("  • Hyperparameter tuning")
print("  • Model selection")
print("  • Performance estimation")

## Step 11: Model Training Example (with Proper Pipeline)

### Example: Random Forest with proper cross-validation

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Train model on training data
model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,  # Control complexity for small data
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

print("Training model on training set only...")
model.fit(X_train_scaled, y_train_transformed)

# Evaluate on each split
for split_name, X_split, y_split in [
    ('Train', X_train_scaled, y_train_transformed),
    ('Validation', X_val_scaled, y_val_transformed),
    ('Test', X_test_scaled, y_test_transformed)
]:
    y_pred = model.predict(X_split)
    rmse = np.sqrt(mean_squared_error(y_split, y_pred))
    mae = mean_absolute_error(y_split, y_pred)
    r2 = r2_score(y_split, y_pred)
    
    print(f"\n{split_name} Set Performance:")
    print(f"   RMSE: {rmse:.3f}")
    print(f"   MAE: {mae:.3f}")
    print(f"   R²: {r2:.3f}")

## Step 12: Applicability Domain Check

### Report which test molecules are within the model's applicability domain

In [ ]:
# Check applicability domain for test set
train_fps = [AllChem.GetMorganFingerprintAsBitVect(
    Chem.MolFromSmiles(smi), 2, 2048) 
    for smi in train_df['Canonical SMILES']]

test_ad_scores = []
for smi in test_df['Canonical SMILES']:
    test_fp = AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(smi), 2, 2048)
    ad_score = processor.estimate_applicability_domain(train_fps, test_fp, k=5)
    test_ad_scores.append(ad_score)

test_df['AD_Score'] = test_ad_scores

# Molecules with AD score < 0.5 are outside applicability domain
within_ad = test_df[test_df['AD_Score'] >= 0.5]
outside_ad = test_df[test_df['AD_Score'] < 0.5]

print(f"\n📊 Applicability Domain Analysis:")
print(f"   Molecules within AD (score ≥ 0.5): {len(within_ad)} ({len(within_ad)/len(test_df)*100:.1f}%)")
print(f"   Molecules outside AD (score < 0.5): {len(outside_ad)} ({len(outside_ad)/len(test_df)*100:.1f}%)")
print(f"\n   Mean AD score: {np.mean(test_ad_scores):.3f}")
print(f"   Median AD score: {np.median(test_ad_scores):.3f}")

## Summary: Data Leakage Checklist ✅

### Before submitting your QSAR model, verify:

- [ ] **Splitting Strategy**
  - Used scaffold-based (not random) splitting
  - Reported scaffold overlap = 0%
  
- [ ] **Duplicates**
  - Canonicalized SMILES
  - Removed exact duplicates before splitting
  - Checked for near-duplicates (Tanimoto ≥ 0.95)
  
- [ ] **Feature Engineering**
  - Fitted scaler/normalizer on TRAIN only
  - Applied transformation to val/test separately
  - No feature selection on full dataset
  
- [ ] **Target Variable**
  - Transformed before splitting OR fitted on train only
  - No target-derived features
  
- [ ] **Cross-Validation**
  - Used scaffold-based K-fold (not random)
  - Nested CV for hyperparameter tuning
  
- [ ] **Reporting**
  - Train-test similarity distribution shown
  - Applicability domain reported
  - Model complexity appropriate for data size
  
- [ ] **External Validation** (if available)
  - Completely unseen compounds
  - No SMILES overlap with training
  - Performance on external set reported

---

### 🎯 Next Steps:
1. Apply this approach to Model 1, 2, 3, and 4
2. Re-train models with proper splitting
3. Compare performance (expect lower but more realistic metrics)
4. Document all leakage prevention steps
5. Prepare for publication/review with proper methodology

## Save Cleaned Data for Other Models

In [ ]:
# Save the splits for use in other models
train_df.to_csv('train_set_no_leakage.csv', index=False)
val_df.to_csv('val_set_no_leakage.csv', index=False)
test_df.to_csv('test_set_no_leakage.csv', index=False)

# Save indices for reproducibility
np.save('train_indices.npy', train_idx)
np.save('val_indices.npy', val_idx)
np.save('test_indices.npy', test_idx)

print("✓ Data saved successfully!")
print("\nUse these files in Models 1, 2, 3, and 4 to ensure consistency.")